📌从`torchvision.datasets`中加载数据集

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import copy
import torch.ao.quantization.quantize_fx as quantize_fx
import os
from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)

In [ ]:
# ---------- 1. 模型定义 ----------
# VGG11 配置
vgg11_cfg = [64, 'M', 128, 'M', 256, 'M', 512, 'M', 512, 'M']

class VGG11(nn.Module):
    def __init__(self):
        super(VGG11, self).__init__()
        self.features = self._make_layers(vgg11_cfg)
        self.classifier = nn.Linear(512, 10)  # CIFAR-10 has 10 classes

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [
                    nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                    nn.BatchNorm2d(x),
                    nn.ReLU(inplace=True)
                ]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [ ]:
import time
import os
from google.colab import drive
drive.mount('/content/drive')
# 用于测试模型精度以及速度的函数
def test(model, test_loader, debug = False):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        start_time = time.time()
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        end_time = time.time()
    accuracy = 100 * correct / total
    time_cost = end_time - start_time
    if debug:
        print('Accuracy of the network on the %d test images: %.2f %%' % (n_test, accuracy))
        print('Time cost: %.2f s' % time_cost)
    return accuracy, time_cost

# 用于获取模型大小的函数，单位为 MB
def get_model_size(model):
    torch.save(model.state_dict(), "temp.pth")
    size = os.path.getsize("temp.pth")/1e6
    os.remove('temp.pth')
    return size

Mounted at /content/drive


In [ ]:
# 加载 CIFAR10 数据集
batch_size = 64

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='/bohr/cifar10-h7hf/v2', train=True, download=True, transform=transform_train)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='/bohr/cifar10-h7hf/v2', train=False, download=True, transform=transform_test)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)
n_test = len(testset)
example_input = (testset[0])

100%|██████████| 170M/170M [00:03<00:00, 43.7MB/s]


In [ ]:
# 加载预训练模型
model_fp = VGG11()
model_fp.load_state_dict(torch.load('/content/drive/MyDrive/第二次尝试/VGG11_CIFAR10.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
model_to_quantize_ptsq = copy.deepcopy(model_fp)
model_to_quantize_ptsq.eval()

qconfig_mapping = get_default_qconfig_mapping("qnnpack")
model_prepared_ptsq = quantize_fx.prepare_fx(model_to_quantize_ptsq, qconfig_mapping, example_input)
# 这里使用全部的训练数据来对模型进行校准
# 注意：校准并不是训练，所以我们不需要对模型做反向传播
model_prepared_ptsq.eval()
with torch.no_grad():
    for images, labels in train_loader:
        model_prepared_ptsq(images)

model_quantized_ptsq = quantize_fx.convert_fx(model_prepared_ptsq)

量化结束，我们开始对`model_quantized_ptsq`模型的卷积层进行剪枝：
1.找到模型的卷积层有哪些
2.使用封装好的方法得到模型卷积层敏感度
3.